In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
import visualkeras



In [ ]:
train_data_dir = '../data/train/'

img_size = (96, 96)
batch_size = 32

# Data generator with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming one-hot encoding for class labels
    color_mode='rgb'
)


In [10]:
# Create and compile the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='elu', input_shape=(96, 96, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='elu'),
    layers.MaxPooling2D((2, 2)),


  
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax') 
])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10
)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 94, 94, 32)        896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 47, 47, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 45, 45, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 22, 22, 32)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 15488)             0         
                                                                 
 dense_9 (Dense)             (None, 128)              

Parameter tuning - Second Version of CNN modified

In [ ]:
# Create and compile the second version of the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='elu', input_shape=(96, 96, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='elu'),
    layers.MaxPooling2D((2, 2)),


  
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax') 
])

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

batch_size = 64 # Batch size updated

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=8
)


Visualize the above CNN model

In [ ]:
visualkeras.layered_view(model)

Plot the train accuracy over the epochs

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
test_data_dir = '../data/test/'

# Image size and batch size
img_size = (96, 96)
batch_size = 32

# Create data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming one-hot encoding for class labels
    color_mode='rgb',
    shuffle=False  # Keep the order of predictions for evaluation
)

# Predict on the test set
predictions = model.predict(test_generator)

# Get predicted classes
predicted_classes = tf.argmax(predictions, axis=1)

# Get true classes
true_classes = test_generator.classes

# Calculate accuracy
test_accuracy = accuracy_score(true_classes, predicted_classes)
print(f'Test Accuracy: {test_accuracy}')

# Generate a classification report
class_report = classification_report(true_classes, predicted_classes)
print('Classification Report:\n', class_report)

Plot the confusion matrices

In [ ]:
# Generate a confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_generator.class_indices,
            yticklabels=test_generator.class_indices)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

Save the trained model

In [ ]:
# Save the trained model
#model.save('./second_cnn_model.h5')